In [1]:
import numpy as np
import pandas as pd
from matplotlib import gridspec
from astropy.io import fits
from astropy.table import Table
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
from glob import glob
import seaborn as sea
import os

sea.set(style='white')
sea.set(style='ticks')
sea.set_style({'xtick.direct'
               'ion': 'in','xtick.top':True,'xtick.minor.visible': True,
               'ytick.direction': "in",'ytick.right': True,'ytick.minor.visible': True})

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
###### REPLACE WITH YOUR FILE LOCATION #######
gnd_EZ = fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodsn_3dhst.v4.4.data.fits')
gsd_EZ = fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodss_3dhst.v4.4.data.fits')

gnd_EZ_cat = Table.read(fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodsn_3dhst.v4.4.zout.fits'),
                    format = 'fits').to_pandas()
gsd_EZ_cat = Table.read(fits.open('/Users/rosaliaobrien/research/website/eazy_catalogs/goodss_3dhst.v4.4.zout.fits'),
                    format = 'fits').to_pandas()

gnd_cat = Table.read('/Users/rosaliaobrien/research/website/catalogs/goodsn_3dhst.v4.4.cat',
                    format = 'ascii').to_pandas()
gsd_cat = Table.read('/Users/rosaliaobrien/research/website/catalogs/goodss_3dhst.v4.4.cat',
                    format = 'ascii').to_pandas()

masterlist = pd.read_pickle('/Users/rosaliaobrien/research/website/eazy_catalogs/templates/master_template_list.pkl')

In [3]:
def Phot_get(field, galaxy_id, masterlist):
#     cat = catalog[catalog.id == galaxy_id]

    if field[1] == 'N':
        pre= 'N_'
        cat = gnd_cat[gnd_cat.id == galaxy_id]

    else:
        pre= 'S_'
        cat = gsd_cat[gsd_cat.id == galaxy_id]

    eff_wv = []
    phot_fl = []
    phot_er = []
    phot_num = []

    for i in cat.keys():
        if i[0:2] == 'f_':
            Clam = 3E18 / masterlist.eff_wv[masterlist.tmp_name == pre + i].values[0] **2 * 10**((-1.1)/2.5-29)
            if cat[i].values[0] > -99.0:
                eff_wv.append(masterlist.eff_wv[masterlist.tmp_name == pre + i].values[0])
                phot_fl.append(cat[i].values[0]*Clam)
                phot_num.append(masterlist.tmp_num[masterlist.tmp_name == pre + i].values[0])
        if i[0:2] == 'e_':
            if cat[i].values[0] > -99.0:
                phot_er.append(cat[i].values[0]*Clam)

    return np.array([eff_wv, phot_fl, phot_er])

def read_templates(wave, temps):
    templates = []

    for temp in temps:
        templ = Template(wave, temp)
        templ.set_fnu()
        templates.append(templ)

    return templates

class Template():
    def __init__(self,tempwv, tempflux):
        self.wave = tempwv
        self.flux = tempflux
        self.flux_fnu = None
        self.set_fnu()

    def set_fnu(self):
        self.flux_fnu = self.flux * self.wave**2 / 3.e18


def full_sed(z, coeffs, templates):
    TEMP =read_templates(template_wv,templates)                      
    NTEMP = len(TEMP)
    
    import astropy.units as u

    templ = TEMP
    tempflux = np.zeros((NTEMP, templ[0].wave.shape[0]))
    for i in range(NTEMP):
        tempflux[i, :] = templ[i].flux_fnu

    templz = templ[0].wave*(1+z)

    templf = np.dot(coeffs, tempflux)
    return templz, templf

In [4]:
templates = gnd_EZ['TEMPF'].data
template_wv =gnd_EZ['TEMPL'].data

In [5]:
def Generate_plot(Field, galaxy_ID):
    
    if Field[1] == 'N':
        coeffs = gnd_EZ['COEFFS'].data[galaxy_ID - 1]
        zgrid = gnd_EZ['ZGRID'].data 
        zchi2 = gnd_EZ['CHI2'].data[galaxy_ID - 1]
        zbest = gnd_EZ_cat.query('id == {0}'.format(galaxy_ID)).z500.values[0]
    
    else:
        coeffs = gsd_EZ['COEFFS'].data[galaxy_ID - 1]
        zgrid = gsd_EZ['ZGRID'].data 
        zchi2 = gsd_EZ['CHI2'].data[galaxy_ID - 1]
        zbest = gsd_EZ_cat.query('id == {0}'.format(galaxy_ID)).z500.values[0]

    ####### get photometry
    effwv, phot, phot_er = Phot_get(Field, galaxy_ID, masterlist)
    
    ####### generate SED
    wv,fl = full_sed(zbest, coeffs, templates)
    fnu_factor = 10**(-0.4*(25.0+48.6))
    flam_spec = 3.e18/wv**2*1E19

    ####### set up grid
    gs = gridspec.GridSpec(1,4, wspace=.3)

    plt.figure(figsize = [20,4])
    
    ####### make flam plot
    plt.subplot(gs[0])
    plt.plot(np.log10(wv*1E-4),fl*fnu_factor*flam_spec, alpha=0.5, label ='z = {0:1.3f}'.format(zbest))
    plt.errorbar(np.log10(effwv*1E-4), phot*1E19, phot_er*1E19, color='r', marker = 'o', linestyle = 'none', 
                 markersize = 7)
    plt.xticks(np.log10([0.5, 1, 2.5, 5.0]),[0.5, 1, 2.5, 5.0])
    plt.legend(fontsize = 10, frameon = True)
    plt.xlabel('Observed Wavelength ($\mu$m)', fontsize=17)
    plt.ylabel('F$_\lambda$ ($10^{-19}$ $erg/s/cm^{2}/\AA $)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=13)
    plt.xlim(np.log10(0.3),np.log10(9))

    ####### make fnu plot
    iscale = interp1d(wv,flam_spec)(effwv)
    fnu = phot*1E19 / iscale
    dfnu = phot_er*1E19/ iscale    
    
    plt.subplot(gs[1])
    plt.plot(np.log10(wv*1E-4),np.log10(fl*fnu_factor), alpha=0.5)
    plt.errorbar(np.log10(effwv*1E-4), np.log10(fnu), dfnu / (fnu * np.log(10)), color='r', marker = 'o', 
                 linestyle = 'none', markersize = 7)
    plt.xticks(np.log10([0.5, 1, 2.5, 5.0]),[0.5, 1, 2.5, 5.0])

    plt.xlabel('Observed Wavelength ($\mu$m)', fontsize=17)
    plt.ylabel('$\log(F_\\nu)$ $\log(10^{23}$ Jy)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=13)
    plt.xlim(np.log10(0.3),np.log10(9))
    plt.ylim(min(np.log10(fnu))*1.02 , max(np.log10(fnu)) * 0.98)

    ####### make chi2 plot
    
    plt.subplot(gs[2])
    plt.plot(zgrid, zchi2)
    plt.xlabel('Redshift (z)', fontsize=17)
    plt.ylabel('$\chi^2$', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=12)

    ####### make P(z) plot
    plt.subplot(gs[3])
    P = np.exp( - np.array(zchi2).astype(np.float128) / 2)
    P /= np.trapz(P,zgrid)

    plt.plot(zgrid, P)
    plt.xlabel('Redshift (z)', fontsize=17)
    plt.ylabel('P(z)', fontsize=17)
    plt.tick_params(axis='both', which='major', labelsize=12)
    #NOTE: they are currently all saving in GOODS_South folder (change after error is fixed)
    plt.savefig('/Users/rosaliaobrien/research/website/eazy_plots/{0}_{1}_EZ.png'.format(Field, galaxy_ID), 
                bbox_inches='tight')

In [6]:
#turn off interactive mode so a million plots dont display while running code
plt.ioff()

subfields = ['GS4', 'ERSPRIME']

for j in subfields:
    df=pd.read_csv('/Users/rosaliaobrien/research/website/website_data/'+j+'.cat', sep=' ')
    gid=df['ids']
    
    #pass objects that give error (have values that are NaN or Inf)
    for i in gid:
        print('Generating plot '+str(j)+'_'+str(i))
        try:
            Generate_plot(j, i)
        except:
            pass

Generating plot GS4_18761
Generating plot GS4_18795
Generating plot GS4_18933
Generating plot GS4_19113
Generating plot GS4_19146
Generating plot GS4_19148
Generating plot GS4_19234


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in log10
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:55: RuntimeWarning: invalid value encountered in log10


Generating plot GS4_19257
Generating plot GS4_19348
Generating plot GS4_19365
Generating plot GS4_19715
Generating plot GS4_19743
Generating plot GS4_19819
Generating plot GS4_19872
Generating plot GS4_19906
Generating plot GS4_19915
Generating plot GS4_19939
Generating plot GS4_19989
Generating plot GS4_19999
Generating plot GS4_20018
Generating plot GS4_20030


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Generating plot GS4_20067
Generating plot GS4_20106
Generating plot GS4_20164
Generating plot GS4_20178
Generating plot GS4_20199
Generating plot GS4_20253
Generating plot GS4_20254
Generating plot GS4_20356
Generating plot GS4_20398
Generating plot GS4_20404
Generating plot GS4_20442
Generating plot GS4_20454
Generating plot GS4_20490
Generating plot GS4_20547
Generating plot GS4_20549
Generating plot GS4_20566
Generating plot GS4_20604
Generating plot GS4_20612
Generating plot GS4_20614
Generating plot GS4_20623
Generating plot GS4_20634
Generating plot GS4_20644
Generating plot GS4_20651
Generating plot GS4_20659
Generating plot GS4_20671
Generating plot GS4_20698
Generating plot GS4_20701
Generating plot GS4_20734
Generating plot GS4_20778
Generating plot GS4_20788
Generating plot GS4_20798
Generating plot GS4_20805
Generating plot GS4_20883
Generating plot GS4_20898
Generating plot GS4_20915
Generating plot GS4_20948
Generating plot GS4_20970
Generating plot GS4_20976
Generating p

Generating plot GS4_28495
Generating plot GS4_28656
Generating plot GS4_28662
Generating plot GS4_28680
Generating plot GS4_28704
Generating plot GS4_28762
Generating plot GS4_28773
Generating plot GS4_28785
Generating plot GS4_28794
Generating plot GS4_28870
Generating plot GS4_28876
Generating plot GS4_28878
Generating plot GS4_28890
Generating plot GS4_28975
Generating plot GS4_28995
Generating plot GS4_29018
Generating plot GS4_29047
Generating plot GS4_29048
Generating plot GS4_29053
Generating plot GS4_29054
Generating plot GS4_29122
Generating plot GS4_29155
Generating plot GS4_29169
Generating plot GS4_29236
Generating plot GS4_29238
Generating plot GS4_29256
Generating plot GS4_29257
Generating plot GS4_29293
Generating plot GS4_29313
Generating plot GS4_29323
Generating plot GS4_29349
Generating plot GS4_29359
Generating plot GS4_29360
Generating plot GS4_29385
Generating plot GS4_29394
Generating plot GS4_29431
Generating plot GS4_29442
Generating plot GS4_29460
Generating p

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: RuntimeWarning: divide by zero encountered in log10


Generating plot GS4_30277
Generating plot GS4_30285
Generating plot GS4_30327
Generating plot GS4_30367
Generating plot GS4_30381
Generating plot GS4_30387
Generating plot GS4_30427
Generating plot GS4_30442
Generating plot GS4_30443
Generating plot GS4_30479
Generating plot GS4_30489
Generating plot GS4_30490
Generating plot GS4_30510
Generating plot GS4_30520
Generating plot GS4_30534
Generating plot GS4_30566
Generating plot GS4_30572
Generating plot GS4_30587
Generating plot GS4_30597
Generating plot GS4_30598
Generating plot GS4_30691
Generating plot GS4_30694
Generating plot GS4_30725
Generating plot GS4_30760
Generating plot GS4_30777
Generating plot GS4_30792
Generating plot GS4_30807
Generating plot GS4_30871
Generating plot GS4_30872
Generating plot GS4_30930
Generating plot GS4_30931
Generating plot GS4_30952
Generating plot GS4_30976
Generating plot GS4_30991
Generating plot GS4_31039
Generating plot GS4_31044
Generating plot GS4_31063
Generating plot GS4_31064
Generating p

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in true_divide


Generating plot ERSPRIME_40676
Generating plot ERSPRIME_40679
Generating plot ERSPRIME_40681
Generating plot ERSPRIME_40703
Generating plot ERSPRIME_40711
Generating plot ERSPRIME_40730
Generating plot ERSPRIME_40759
Generating plot ERSPRIME_40762
Generating plot ERSPRIME_40768
Generating plot ERSPRIME_40776
Generating plot ERSPRIME_40786
Generating plot ERSPRIME_40826
Generating plot ERSPRIME_40829
Generating plot ERSPRIME_40838
Generating plot ERSPRIME_40846
Generating plot ERSPRIME_40850
Generating plot ERSPRIME_40862
Generating plot ERSPRIME_40909
Generating plot ERSPRIME_40956
Generating plot ERSPRIME_40985
Generating plot ERSPRIME_40993
Generating plot ERSPRIME_41095
Generating plot ERSPRIME_41115
Generating plot ERSPRIME_41125
Generating plot ERSPRIME_41127
Generating plot ERSPRIME_41146
Generating plot ERSPRIME_41147
Generating plot ERSPRIME_41151
Generating plot ERSPRIME_41192
Generating plot ERSPRIME_41194
Generating plot ERSPRIME_41264
Generating plot ERSPRIME_41266
Generati

In [7]:
print('DONE')

DONE
